In [1]:
# Run this code cell without changes to load the training and validation sets

# For simplicity, we drop categorical variables

import pandas as pd
from sklearn.model_selection import train_test_split

#  Read the data
train_data = pd.read_csv('../../datasets/home-data-for-ml-course/train.csv', index_col='Id')
test_data = pd.read_csv('../../datasets/home-data-for-ml-course/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = train_data.SalePrice
train_data.drop(['SalePrice'], axis=1, inplace=True)

# Select numeric columns only
numeric_cols = [cname for cname in train_data.columns if
                train_data[cname].dtype in ['int64', 'float64']]
X = train_data[numeric_cols].copy()
X_test = test_data[numeric_cols].copy()

In [2]:
# Use this code cell to print first several rows of the data
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,548,0,61,0,0,0,0,0,2,2008
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,460,298,0,0,0,0,0,0,5,2007
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,608,0,42,0,0,0,0,0,9,2008
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,642,0,35,272,0,0,0,0,2,2006
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,836,192,84,0,0,0,0,0,12,2008


In [3]:
'''
    So far, you've learned how to build pipelines with scikit-learn. For instance, the
    pipeline below will use SimpleImputer() to replace missing values in the data, before
    using RandomForestRegressor() to train a random forest model to make predictions. We
    set the number of trees in the random forest model with the n_estimators parameter,
    and setting random_state ensures reproducibility.
'''

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators=50, random_state=0))
])

In [4]:
'''
    You have also learned how to use pipelines in cross-validation. The code below uses
    the cross_val_score() function to obtain the mean absolute error (MAE), averaged
    across five different folds. Recall we set the number of folds with the cv parameter.
'''
from sklearn.model_selection import cross_val_score
# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
print("Average MAE score:", scores.mean())

Average MAE score: 18276.410356164386


# Step 1:Write a useful function
In this exercise, you'll use cross-validation to select parameters
for a machine learning model.

Begin by writing a function get_score() that reports the average (over
three cross-validation folds) MAE of a machine learning pipeline that uses:

    1. the data in X and y to create folds,
    2. SimpleImputer() (with all parameters left as default) to replace missing
    values, and
    3. RandomForestRegressor() (with random_state=0) to fit a random forest
    model.

    The n_estimators parameters supplied to get_score() is used when setting
    the number of trees in the random forest model.

In [5]:
def get_score(n_estimators):
    """
        return: the average MAE over 3 CV folds of random forest model
        Keyword argument:
        n_estimators -- the number of trees in the forest.
    """
    # Replace this body with your own code
    pipeline = Pipeline(steps=[
        ('preprocessor', SimpleImputer()),
        ('model', RandomForestRegressor(n_estimators=n_estimators,
                                        random_state=0))
    ])
    scores = -1 * cross_val_score(my_pipeline, X, y,
                                  cv=5,
                                  scoring='neg_mean_absolute_error')
    return scores.mean()

# Step 2: Test different parameter values

Now, you will use the function that you defined in Step 1 to evaluate the
model performance corresponding to eight different values for the number of
trees in the random forest: 50, 100, 150, ..., 300, 350, 400.

Store your results in a Python dictionary results, where results[i] is the
average MAE returned by get_score(i).
